In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re

In [26]:
positive_count, negative_count=0,0
positive_corpus,negative_corpus=[],[]
total=0
with open('IMDB Dataset.csv','r') as csv_file:
    csv_reader=csv.reader(csv_file)
    for line in csv_reader:
        total+=1
        if total==1394: #something wrong with the other line
            break
        else:
            if line[1]=='positive':
                positive_count+=1
                positive_corpus.append(line[0])
            elif line[1]=='negative':
                negative_count+=1
                negative_corpus.append(line[0])
print(positive_count,negative_count)

704 688


In [43]:
def totalWords(corpus):
    words=0
    for i in corpus:
        words+=len(i.split(" "))
    return words
totalWords(positive_corpus),totalWords(negative_corpus)

(161387, 150354)

In [44]:
totalWordspositive=totalWords(positive_corpus)
totalWordsnegative=totalWords(negative_corpus)

In [45]:
total_corpus_count=positive_count+negative_count
probabilities=dict()
probabilities["positive"]=positive_count/total_corpus_count
probabilities["negative"]=negative_count/total_corpus_count
probabilities

{'positive': 0.5057471264367817, 'negative': 0.4942528735632184}

In [46]:
ref_words=[]
with open('positive.csv') as pos:
    pos_reader=csv.reader(pos)
    for line in pos_reader:
        ref_words.append(line[0])
        
with open('negative.csv') as neg:
    neg_reader=csv.reader(neg)
    for line in neg_reader:
        ref_words.append(line[0])

In [47]:
np.array(ref_words)

array(['a+', 'abound', 'abounds', ..., 'glad', 'yum', 'interesting'],
      dtype='<U20')

In [59]:
def parameter_estimation(probabilities,ref_words,totalWordspositive,totalWordsnegative):
    #probabilities refers to a dictionary of given probabilities
    #ref words (in the name itself)
    #Compute P(Word | pos) and P(Word | neg)
    for word in ref_words:
        count=0
        for texts in positive_corpus:
            count+=len(re.findall(word,texts))
        probabilities[word+" | positive"]=(count+1)/totalWordspositive
        if count==0:
            probabilities[word+" | positive"]=1/totalWordspositive
        count=0
        for texts in negative_corpus:
            count+=len(re.findall(word,texts))
        probabilities[word+" | negative"]=(count+1)/totalWordsnegative
        if count==0:
            probabilities[word+" | negative"]=1/totalWordsnegative
    return probabilities
            
prob_dict=parameter_estimation(probabilities,ref_words,totalWordspositive,totalWordsnegative)        

In [60]:
for prob in prob_dict:
    if prob_dict[prob]>1:
        print(prob_dict[prob])
        print(prob)
    if prob_dict[prob]==0.0:
        print(prob_dict[prob])
        print(prob)

In [61]:
prob_dict

{'positive': 0.5057471264367817,
 'negative': 0.4942528735632184,
 'a+ | positive': 0.34795863359502316,
 'a+ | negative': 0.33641273261768895,
 'abound | positive': 1.239257189240769e-05,
 'abound | negative': 3.3254851882889714e-05,
 'abounds | positive': 6.196285946203845e-06,
 'abounds | negative': 6.650970376577943e-06,
 'abundance | positive': 1.8588857838611536e-05,
 'abundance | negative': 2.660388150631177e-05,
 'abundant | positive': 1.239257189240769e-05,
 'abundant | negative': 1.995291112973383e-05,
 'accessable | positive': 6.196285946203845e-06,
 'accessable | negative': 6.650970376577943e-06,
 'accessible | positive': 6.196285946203845e-06,
 'accessible | negative': 1.3301940753155886e-05,
 'acclaim | positive': 3.098142973101923e-05,
 'acclaim | negative': 1.3301940753155886e-05,
 'acclaimed | positive': 2.478514378481538e-05,
 'acclaimed | negative': 6.650970376577943e-06,
 'acclamation | positive': 6.196285946203845e-06,
 'acclamation | negative': 6.650970376577943e-

In [64]:
def NaiveBayesMultinomial(prob_dict,new_text,ref_words):
    new_text=new_text.split(" ")
    positive_score,negative_score=prob_dict["positive"],prob_dict["negative"]
    for everyword in new_text:
        if everyword in ref_words:
            positive_score*=prob_dict[everyword+" | positive"]
            negative_score*=prob_dict[everyword+" | negative"]
    if negative_score > positive_score:
        print("It is negative")
    elif positive_score>negative_score:
        print("It is positive")
    else:
        print("It is a moderate comment")
    print(positive_score,negative_score)

In [65]:
NaiveBayesMultinomial(prob_dict,"I love this movie to death. OMG I cannot wait to watch it again soon !",ref_words)

It is positive
0.0010811450650962573 0.00041419491379654364


In [66]:
NaiveBayesMultinomial(prob_dict,"This movie is the tragically the not-so good and thus the worst shit I have ever had to feast my eyes on",ref_words)

It is negative
0.0011814251870762 0.0012425847413896308


In [67]:
def Gaussian(mean,variance,x):
    return (1/np.sqrt(2*np.pi*variance))*np.exp(-0.5*((x-mean)/np.sqrt(variance))**2)

In [81]:
male,female=[],[]
with open('human_data.csv') as pos:
    pos_reader=csv.reader(pos)
    count=-1
    for line in pos_reader:
        count+=1
        if line[0].lower()=="male":
            male.append([line[1],line[2]])
        elif line[0].lower()=="female":
            female.append([line[1],line[2]])

In [86]:
male=np.array(male,dtype="float")
female=np.array(female,dtype="float")

In [94]:
means=np.array([np.mean(male[:,0]),np.mean(male[:,1]),np.mean(female[:,0]),np.mean(female[:,1])])
stdev=np.array([np.std(male[:,0]),np.std(male[:,1]),np.std(female[:,0]),np.std(female[:,1])])
print(means)
print(stdev)
parameters=np.array([means,stdev])
parameters.shape

[ 69.02634591 187.02062066  63.7087736  135.86009301]
[ 2.86307588 19.7791763   2.69601437 19.02056546]


(2, 4)

In [135]:
def NaiveBayesGaussian(parameters,new_data):
    likelihood_vector=Gaussian(means,stdev**2,new_data)
    logs=np.log(likelihood_vector)
    male=np.sum(logs[:2],keepdims=True)
    female=np.sum(logs[2:],keepdims=True)
    print("Male Score: ", male)
    print("Female Score:", female)
    if male>female:
        print("Subject with {} kg weight and {} cm height is most likely a male".format(new_data[0],new_data[1]))
        return 1
    else:
        print("Subject with {} kg weight and {} cm height is most likely a female".format(new_data[0],new_data[1]))
        return 0
NaiveBayesGaussian(parameters,np.array([88,152,88,152]))

Male Score:  [-29.40056145]
Female Score: [-46.72569868]
Subject with 88 kg weight and 152 cm height is most likely a male


1

In [150]:
test_data=np.zeros((337,3))
with open('test_data_human_dim.csv') as tester:
    reader=csv.reader(tester)
    count=0
    for line in reader:
        if float(line[2])<20.0:
            continue
        else:
            test_data[count,0:3]=line[1],line[0],line[3]
            count+=1
print(count)
    
test_data

337


array([[ 47.8256065, 151.765    ,   1.       ],
       [ 36.4858065, 139.7      ,   0.       ],
       [ 31.864838 , 136.525    ,   0.       ],
       ...,
       [ 52.16308  , 162.56     ,   1.       ],
       [ 54.0624965, 156.21     ,   0.       ],
       [ 52.5316235, 158.75     ,   1.       ]])

In [151]:
y_bar=test_data[:,2].reshape(337,1)
y_bar

array([[1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],

In [152]:
def create_array(test_data):
    new_array=np.zeros((337,4))
    count=0
    for row in test_data:
        new_array[count,:]=row[0],row[1],row[0],row[1]
        count+=1
    return new_array
formatted_data=create_array(test_data)

In [153]:
formatted_data

array([[ 47.8256065, 151.765    ,  47.8256065, 151.765    ],
       [ 36.4858065, 139.7      ,  36.4858065, 139.7      ],
       [ 31.864838 , 136.525    ,  31.864838 , 136.525    ],
       ...,
       [ 52.16308  , 162.56     ,  52.16308  , 162.56     ],
       [ 54.0624965, 156.21     ,  54.0624965, 156.21     ],
       [ 52.5316235, 158.75     ,  52.5316235, 158.75     ]])

In [189]:
y_predicted=np.zeros((337,1))
row_number=0
for row in formatted_data:
    y_predicted[row_number,:][0]=NaiveBayesGaussian(parameters,row)
    if y_predicted[row_number,:]==1:
        print("male found")
    row_number+=1
#print(y_predicted)

Male Score:  [-34.87912749]
Female Score: [-23.47879223]
Subject with 47.8256065 kg weight and 151.765 cm height is most likely a female
Male Score:  [-73.32459294]
Female Score: [-56.77516118]
Subject with 36.4858065 kg weight and 139.7 cm height is most likely a female
Male Score:  [-93.36791096]
Female Score: [-75.53138227]
Subject with 31.864838 kg weight and 136.525 cm height is most likely a female
Male Score:  [-22.62287836]
Female Score: [-14.21084075]
Subject with 53.0419145 kg weight and 156.845 cm height is most likely a female
Male Score:  [-55.05604097]
Female Score: [-40.51582567]
Subject with 41.276872 kg weight and 145.415 cm height is most likely a female
Male Score:  [-8.78240286]
Female Score: [-6.89165411]
Subject with 62.992589 kg weight and 163.83 cm height is most likely a female
Male Score:  [-65.49942502]
Female Score: [-50.63110046]
Subject with 38.2434755 kg weight and 149.225 cm height is most likely a female
Male Score:  [-17.48670371]
Female Score: [-11.94

In [184]:
NaiveBayesGaussian(parameters,np.array([62.99,163.83,62.99,163.83]))

Male Score:  [-8.78430897]
Female Score: [-6.89190968]
Subject with 62.99 kg weight and 163.83 cm height is most likely a female


0

In [185]:
difference_vector=y_bar-y_predicted

In [186]:
difference_vector.shape

(337, 1)

In [188]:
count=0
for i in range(0,337):
    if difference_vector[i,:][0]!=0:
        count+=1
print(count)

160


In [191]:
percentage_error=count*100/337
percentage_error

47.47774480712166